In [1]:
import pandas as pd
import csv
import itertools
from sklearn.linear_model import LogisticRegression
import numpy as np
import re
import csv
import Levenshtein as lev
from fuzzywuzzy import fuzz

## 1. Data Reading

In [2]:
# Read in the data as pandas dataframes
train_df = pd.read_csv("train.csv")
rtable_df = pd.read_csv("rtable.csv")
ltable_df = pd.read_csv("ltable.csv")

rtable_df.rename(columns = {"id": "r_id", "title": "r_title", "category": "r_category", "brand":"r_brand", "modelno":"r_modelno","price":"r_price"}, inplace = True)
ltable_df.rename(columns = {"id": "l_id", "title": "l_title", "category": "l_category", "brand":"l_brand", "modelno":"l_modelno","price":"l_price"}, inplace = True)

In [3]:
def training_data(train_df):
    training_set = []
    labels = []
    for row_num in range(len(train_df)): # row_num is equal to id number
        i = train_df.iloc[row_num,:].loc["ltable_id"]
        j = train_df.iloc[row_num,:].loc["rtable_id"]
        k = train_df.iloc[row_num,:].loc["label"]
        training_set.append((i,j))
        labels.append(k)
    return training_set, labels

training_set, labels = training_data(train_df)

In [4]:
def add2training(training_set, ltable, rtable):
 
    ltable_ids = []
    rtable_ids = []
    for i,j in training_set: # each tuple pair
        ltable_ids.append(i)
        rtable_ids.append(j)

    ltable = ltable.loc[ltable_ids,:]
    rtable = rtable.loc[rtable_ids,:]

    ltable.reset_index(inplace=True)
    rtable.reset_index(inplace=True)

    training_df = pd.concat([ltable, rtable], axis = 1)
    return training_df

training_df = add2training(training_set, ltable_df, rtable_df)

## 2. Blocking Step

In [5]:
rtable_df["key"] = 0
ltable_df["key"] = 0
df_cartesian = rtable_df.merge(ltable_df, how = "outer") # cartesian product on the 2 dataframes to get all possible combinations and their corresponding attribute info

In [6]:
df_cartesian.drop(columns = ["key"],inplace = True)
df_cartesian["Keep"] = "No" # create a new column to determine whether or not to keep row

In [7]:
df_cartesian['r_brand'] = df_cartesian['r_brand'].astype(str)
df_cartesian['l_brand'] = df_cartesian['l_brand'].astype(str)

# Make sure all brand names are lower case
df_cartesian["r_brand"] = df_cartesian["r_brand"].str.lower()
df_cartesian["l_brand"] = df_cartesian["l_brand"].str.lower()

In [8]:
remove_from_brand = [" ", "-"]
for i in remove_from_brand:
    # remove empty spaces and punctuation in brand names
    df_cartesian["l_brand"] = df_cartesian["l_brand"].apply(lambda x: str(x).replace(i,""))
    df_cartesian["r_brand"] = df_cartesian["r_brand"].apply(lambda x: str(x).replace(i,""))

In [9]:
mask = df_cartesian["l_brand"] == df_cartesian["r_brand"]
df_cartesian.loc[mask,"Keep"] = "Yes" # if brand names are identical (character by character, word for word), change "No" in Keep column to "Yes"

In [10]:
candidate_df = df_cartesian[df_cartesian["Keep"] == "Yes"] # keep only the rows where the brand name is the same
candidate_df.set_index(["l_id","r_id"], inplace = True) # create a multi-index
cand_set = candidate_df.index # number of combinations reduced to 258,219

In [11]:
candidate_df.reset_index(inplace = True)
candidate_df.drop(["Keep"], axis = 1, inplace = True) # drop the "Keep" column because not needed for features section

/Users/daisyzhou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## 3. Feature Engineering

In [12]:
def fuzz_ratio(row,i):
    fuzz_l = row["l_" + i].lower()
    fuzz_r = row["r_" + i].lower()
    fuzz_num = fuzz.token_set_ratio(fuzz_l,fuzz_r)
    return fuzz_num

def levenshtein_dist(row,i):
    ld_l = row["l_" + i].lower()
    ld_r = row["r_" + i].lower()
    lev_dist = lev.distance(ld_l,ld_r)
    return lev_dist

def jaccard_similarity(row,i):
    js_l = row["l_" + i].lower().split()
    js_r = row["r_" + i].lower().split()
    numer = len(set(js_l).intersection(set(js_r)))
    jaccard_num =  numer/max(len(js_l),len(js_r))
    return jaccard_num

def rate_num(row,i): # matching the numbers in the string for each attribute
    str_l = row["l_" + i].lower()
    str_r = row["r_" + i].lower()
    num_l = re.findall(r"[0-9]+",str_l)
    num_r = re.findall(r"[0-9]+",str_r)
    if len(num_l) == 0 & len(num_r) == 0:
        rate = 0
    else:
        rate = len(set(num_l).intersection(set(num_r)))/len(set(num_l).union(set(num_r)))
    return rate
    
def feature_engr(df4feature):
    df4feature = df4feature.astype(str)
    attributes = ["title", "category", "brand", "modelno", "price"]
    feature_vals = []
    for i in attributes:
        jaccard_num = df4feature.apply(jaccard_similarity, i = i, axis = 1)
        lev_dist = df4feature.apply(levenshtein_dist, i = i, axis = 1)
        fuzz_num = df4feature.apply(fuzz_ratio, i = i, axis = 1)
        match_nums = df4feature.apply(rate_num, i = i, axis = 1)
        feature_vals.append(jaccard_num)
        feature_vals.append(lev_dist)
        feature_vals.append(fuzz_num)
        feature_vals.append(match_nums)

    features_arr = np.array(feature_vals).T
    return features_arr

candidate_features = feature_engr(candidate_df)
training_features = feature_engr(training_df)

## 4. Model Training

In [26]:
# Train model and predict, using Logistic Regression classification model
clf = LogisticRegression(class_weight = "balanced",random_state = 0, max_iter = 100000)
clf.fit(training_features,labels)
y_pred = clf.predict(candidate_features)
# number of matches is 2316

## 5. Generating Output and Writing to CSV

In [29]:
# Convert matches to list of tuples
cand_tuples = []
model_matches = candidate_df.loc[y_pred == 1, ["l_id", "r_id"]]
cand_tuples = [(row.l_id, row.r_id) for row in model_matches.itertuples()] # get the (l_id, r_id) pairs where there is a match from running the model

In [30]:
# Get the matches from training set
matched_entries = train_df[train_df["label"] == 1]
matched_tups = [(i.ltable_id, i.rtable_id) for i in matched_entries.itertuples()]

In [31]:
# Remove matches in result if they appeared as a match in the training set
cand_tuples = set(cand_tuples)
for tup in matched_tups:
    if tup in cand_tuples:
        cand_tuples.remove(tup) # number of matches left: 2136 --> 1801


In [32]:
# Write matches from Logistic Regression model to CSV file
with open("MLproject_output.csv", "w") as fout:
    writer = csv.writer(fout, delimiter = ",")
    writer.writerow(["ltable_id","rtable_id"])
    for tup in cand_tuples:
        writer.writerow(tup)